In [1]:
import pandas as pd

In [2]:
reporting_plans = pd.read_parquet('../data/reporting_plans.parquet')

In [3]:
reporting_plans = reporting_plans.fillna('')

In [4]:
ppo_plans = reporting_plans[reporting_plans.plan_name.str.contains('PPO')]

In [5]:
ppo_plans[ppo_plans.plan_name.str.contains('NEW YORK')]
# interesting, but all of these have NY in them. 
# So we can just check for NY since itl ooks like only the business name has NEW YORK

,plan_name,plan_id_type,plan_id,plan_market_type
8499,PPO NY - NEW YORK TECH BENEFITS PROGRAM - ANTHEM,EIN,474780832,group
9833,PPO NY - NEW YORK TECH BENEFITS PROGRAM - ANTHEM,EIN,464876260,group
10488,PPO NY - NEW YORK TECH BENEFITS PROGRAM - ANTHEM,EIN,464131697,group
30364,BLUE ACCESS PPO NY - NEW YORK STATE NURSES ASS...,EIN,237336001,group
41379,PPO WITH HSA NY - NEW YORK TECH BENEFITS PROGR...,EIN,454466563,group
46555,PPO WITH HSA NY - NEW YORK TECH BENEFITS PROGR...,EIN,300855824,group
59986,PPO WITH HSA NY - PLANNED PARENTHOOD OF GREATE...,EIN,132621497,group
71984,PPO WITH HSA NY - GREATER NEW YORK HOSPITAL AS...,EIN,133636375,group
72136,PPO WITH HSA NY - NEW YORK TECH BENEFITS PROGR...,EIN,161778334,group
73851,PPO NY - NEW YORK TECH BENEFITS PROGRAM - ANTHEM,EIN,384033378,group


In [6]:
# I got false positives on NY being part of COMPANY when just checking for NY containment-wise
# Also maybe other words could have NY in them?
# anyway that's the reason for the complexity below
# it shouldn't end with NY because all of these end with "ANTHEM"

ny_ppo_plans = ppo_plans[ppo_plans.plan_name.str.contains(' NY ') | ppo_plans.plan_name.str.startswith('NY')]

In [7]:
ny_ppo_plans

,plan_name,plan_id_type,plan_id,plan_market_type
63,NY SG PPO NETWORK - GLOBAL COMMUNITY CHARTER -...,EIN,453217621,group
96,PPO NY - AUTEL. US INC. - ANTHEM,EIN,452320063,group
333,BLUE ACCESS PPO NY - ZIPARI INC. - ANTHEM,EIN,320435609,group
1143,NY SG PPO NETWORK - FOXPORT LLC - ANTHEM,EIN,815135520,group
1785,NY SG PPO NETWORK - SUPERNOVA IO INC - ANTHEM,EIN,832789008,group
...,...,...,...,...
147545,PPO WITH HSA NY - VETTAFI HOLDINGS LLC - ANTHEM,EIN,830723306,group
148095,PPO EPO FOR NY - HORIZON FOOD EQUIPMENT INC - ...,EIN,141725543,group
148866,NY SG PPO NETWORK - LERMAN MANAGEMENT LLC - AN...,EIN,830914808,group
149738,NY SG PPO NETWORK - M AND S IMPORT CORP - ANTHEM,EIN,271432295,group


In [8]:
# Based on this it looks like they are following a pattern where
# - the first part of "plan_name" is the actual name of the plan
# - the second part of the "plan_name" is the company they are working with
# - the third part of the "plan_name" is...always Anthem?
# And these are all delimited by " - "
# Let's explode it into columns.
# ny_ppo_plans[['plan_name', 'company', 'insurance_company']] = ny_ppo_plans['plan_name'].str.split(' - ',expand=True)
# oh no, that didn't work.

In [9]:
ny_ppo_plans[ny_ppo_plans.plan_name.str.contains('PPO NY')]

,plan_name,plan_id_type,plan_id,plan_market_type
96,PPO NY - AUTEL. US INC. - ANTHEM,EIN,452320063,group
333,BLUE ACCESS PPO NY - ZIPARI INC. - ANTHEM,EIN,320435609,group
5989,BLUE ACCESS PPO NY - NORTH SHORE ANIMAL LEAGUE...,EIN,111666852,group
8499,PPO NY - NEW YORK TECH BENEFITS PROGRAM - ANTHEM,EIN,474780832,group
9833,PPO NY - NEW YORK TECH BENEFITS PROGRAM - ANTHEM,EIN,464876260,group
...,...,...,...,...
140439,PPO NY - BASIN HOLDINGS LLC - ANTHEM,EIN,452496821,group
143300,"PPO NY - GRAMERCY GROUP, INC. - ANTHEM",EIN,133546842,group
143637,PPO NY - MAGNOLIA OPERATING LLC - ANTHEM,EIN,611509168,group
143940,"PPO NY - K HEALTH, INC. - ANTHEM",EIN,814173541,group


In [10]:
ny_ppo_plans[ny_ppo_plans.plan_name.str.contains('PPO NY') & (~ny_ppo_plans.plan_name.str.startswith('PPO NY'))]

,plan_name,plan_id_type,plan_id,plan_market_type
333,BLUE ACCESS PPO NY - ZIPARI INC. - ANTHEM,EIN,320435609,group
5989,BLUE ACCESS PPO NY - NORTH SHORE ANIMAL LEAGUE...,EIN,111666852,group
18843,BLUE ACCESS PPO NY - CHS RECOVERY PARTNERS - A...,EIN,384042589,group
19816,"BLUE ACCESS PPO NY - LOAR GROUP, INC. - ANTHEM",EIN,454104058,group
30364,BLUE ACCESS PPO NY - NEW YORK STATE NURSES ASS...,EIN,237336001,group
33447,BLUE ACCESS PPO NY - BBVA SA - ANTHEM,EIN,133491492,group
54359,BLUE ACCESS PPO NY - MINDS & ASSEMBLY LLC - AN...,EIN,352544538,group
58939,BLUE ACCESS PPO NY - ATRIUM STAFFING LLC - ANTHEM,EIN,133957708,group
61717,BLUE ACCESS PPO NY - ST. BARNABAS HOSPITAL - A...,EIN,131740122,group
63398,BLUE ACCESS PPO NY - THIRD BRIDGE US INC - ANTHEM,EIN,273035072,group


In [11]:
# Based on the EDA above I'm assuming that plans that *start with* PPO NY are the ones we want
# since based on my cursory research the Blue Access plan is different. But I could be wrong!

In [12]:
anthem_ny_ppo_plans = ny_ppo_plans[ny_ppo_plans.plan_name.str.startswith('PPO NY')]

In [13]:
anthem_ny_ppo_plans

,plan_name,plan_id_type,plan_id,plan_market_type
96,PPO NY - AUTEL. US INC. - ANTHEM,EIN,452320063,group
8499,PPO NY - NEW YORK TECH BENEFITS PROGRAM - ANTHEM,EIN,474780832,group
9833,PPO NY - NEW YORK TECH BENEFITS PROGRAM - ANTHEM,EIN,464876260,group
10488,PPO NY - NEW YORK TECH BENEFITS PROGRAM - ANTHEM,EIN,464131697,group
11603,PPO NY - METROPOLITAN COUNCIL ON JEWISH POVERT...,EIN,132738818,group
12381,PPO NY - HUDSON MX INC - ANTHEM,EIN,813652030,group
12612,PPO NY - GUILD TIMES BENEFITS FUND - ANTHEM,EIN,133094045,group
17938,PPO NY - DUTCHESS EDUCATION HEALTH INSURANCE C...,EIN,146001632,group
23661,PPO NY - SAINT ANN'S SCHOOL - ANTHEM,EIN,112606681,group
29864,PPO NY - OASIS OUTSOURCING HOLDINGS INC. - ANTHEM,EIN,650731524,group


In [14]:
# grab the plan IDs so we can use them to extract the MRFs
# anthem_ny_ppo_plans.plan_id.unique()
# I originally did this approach, but then I realized these IDs are EINs, so conceivably they could be associated
# with *many* plans, because the business is probably paying for multiple plans! So we have to go by plan name.

In [15]:
anthem_ny_ppo_plans.plan_name.unique()

array(['PPO NY - AUTEL. US INC. - ANTHEM',
       'PPO NY - NEW YORK TECH BENEFITS PROGRAM - ANTHEM',
       'PPO NY - METROPOLITAN COUNCIL ON JEWISH POVERTY - ANTHEM',
       'PPO NY - HUDSON MX INC - ANTHEM',
       'PPO NY - GUILD TIMES BENEFITS FUND - ANTHEM',
       'PPO NY - DUTCHESS EDUCATION HEALTH INSURANCE CONSORTIUM - ANTHEM',
       "PPO NY - SAINT ANN'S SCHOOL - ANTHEM",
       'PPO NY - OASIS OUTSOURCING HOLDINGS INC. - ANTHEM',
       'PPO NY - CEWW HEALTH INSURANCE CONSORTIUM - ANTHEM',
       'PPO NY - UNITED FEDERATION OF TEACHERS STAFF & WELFARE PLAN - ANTHEM',
       'PPO NY - LYRA TECHNOLOGIES, INC - DBA BLOCK RENOVATION - ANTHEM',
       'PPO NY - COMMUNITY FEDERAL SAVINGS BANK - ANTHEM',
       'PPO NY - MANHATTAN WEST LLC - ANTHEM',
       'PPO NY - BALDOR SPECIALTY FOODS INC - ANTHEM',
       'PPO NY - THE BERKSHIRE BANK - ANTHEM',
       'PPO NY - OVERTIME SPORTS, INC - ANTHEM',
       'PPO NY - DATAMINR INC - ANTHEM',
       'PPO NY - RESTORIXHEALTH LLC - ANT

In [16]:
# Below this point I tried to figure out if I could parse the full_plan_name column consistently into 
# delimited fields following some sort of rule / structure to decide which parts where the plan name.
# I ended up stopping as it was outside the bounds of the take home

In [17]:
exploded_full_plan_names = reporting_plans_full['full_plan_name'].str.split(' - ',expand=True)
exploded_full_plan_names[~exploded_full_plan_names[3].isna()]

NameError: name 'reporting_plans_full' is not defined

In [ ]:
exploded_full_plan_names = reporting_plans_full['full_plan_name'].str.split(' - ',expand=True)
exploded_full_plan_names[~exploded_full_plan_names[4].isna()]

In [ ]:
reporting_plans_full = reporting_plans.assign(full_plan_name=reporting_plans['plan_name'])

def map_split_to_plan_name_company_insurance_company(plan_name):
    name_elements = plan_name.split(' - ')
    plan_name = name_elements[0]  # plan name will always have at least some of name_elements

reporting_plans_full[['plan_name', 'company', 'insurance_company']] = \
    reporting_plans_full['full_plan_name'].str.split(' - ',expand=True)